# Proceso de ETL

### • Puntos a tener en cuenta:

El script debe poder recibir nuevas tablas y arrojar un resultado unificado.

La carga incremental debe estar administrada por el mismo script.

Al data engineer no le importan los outliers, eso es cuestión del analista.

DER: Diagrama de Entidad Relacional

Sobre la carga: habrá una carga más.. No debe relacionarse al tiempo

##### Los problemas que no se alcancen a solucionar hay que señalarlos:

- Los precios en una de las hojas de excel tienen un decimal corrido

#### Crear tablas en md para mostrar valores nulos y duplicados en tablas originales

Entrega: jueves 18hrs argentina (4pm méxico)

In [1]:
# Empezamos importando las librerías que usaremos para la preparación de los datos
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import sqlite3
from datetime import datetime
#import re
#import os
from os import listdir
from random import sample

## Creación de los DataFrames en pandas
Importamos los datasets de diferentes formatos a DataFrames de pandas de manera automatizada, organizándolos en un diccionario para facilitar su acceso y limpieza.

In [2]:
# Obtenemos una lista de los archivos dentro del directorio 'datasets'
file_names = listdir('.\datasets')
print(file_names)

['precios_semanas_20200419_20200426.xlsx', 'precios_semana_20200413.csv', 'precios_semana_20200503.json', 'precios_semana_20200518.txt', 'producto.parquet', 'sucursal.csv']


In [3]:
# Creamos un diccionario con los nombres de cada archivo y su extensión
datasets_extensions = {}
for x in file_names:
    datasets_extensions[x] = x.split('.')

for x in datasets_extensions:
    print(f'\nKey: {x}\nValue: {datasets_extensions[x]}\n')


Key: precios_semanas_20200419_20200426.xlsx
Value: ['precios_semanas_20200419_20200426', 'xlsx']


Key: precios_semana_20200413.csv
Value: ['precios_semana_20200413', 'csv']


Key: precios_semana_20200503.json
Value: ['precios_semana_20200503', 'json']


Key: precios_semana_20200518.txt
Value: ['precios_semana_20200518', 'txt']


Key: producto.parquet
Value: ['producto', 'parquet']


Key: sucursal.csv
Value: ['sucursal', 'csv']



Ahora usaremos las extensiones y los nombres de los archivos para importar los datasets en objetos 'DataFrame' de pandas.

Habiendo estudiado un poco la naturaleza de los datasets antes de importarlos, se pudo observar que los registros
o *tablas de hecho* son las que llevan por nombre 'precios_...', mientras que las *dimensiones* son los datasets de 
'producto' y 'sucursal', pues proveen información adicional a las primeras tablas.

Así, los datasets de precios serán organizados en el diccionario 'ps_2020' (precio semanal 2020) y los otros dos en el 
diccionario 'dims' (dimensiones).

In [4]:
# Creamos los diccionarios y automatizamos la importación de los archivos contenidos en el directorio 'datasets'
ps_2020 = {}
dims = {}
for x in datasets_extensions:
    path = f'datasets/{x}'
    if x[:7] == 'precios':
        if datasets_extensions[x][1] in ['xlsx', 'xls']:
            xl_dict = pd.read_excel(path, sheet_name=None, date_parser=None)
            for sheet in xl_dict:
                name = f'{sheet[-8:-4]}-{sheet[-4:-2]}-{sheet[-2:]}'
                ps_2020[name] = pd.DataFrame(xl_dict[sheet])
        else:
            name = f'{datasets_extensions[x][0][-8:-4]}-{datasets_extensions[x][0][-4:-2]}-{datasets_extensions[x][0][-2:]}'
            if datasets_extensions[x][1] == 'csv':
                ps_2020[name] = pd.read_csv(path, encoding='UTF-16 LE')
            elif datasets_extensions[x][1] == 'json':
                ps_2020[name] = pd.read_json(path)
            elif datasets_extensions[x][1] == 'txt':
                ps_2020[name] = pd.read_csv(path, delimiter='|')
    else:
        name = datasets_extensions[x][0]
        if datasets_extensions[x][1] == 'csv':
            dims[name] = pd.read_csv(path)
        elif datasets_extensions[x][1] == 'parquet':
            dims[name] = pd.read_parquet(path)


In [5]:
for x in ps_2020:
    print(f'• DataFrame: {x}\n• Shape: {ps_2020[x].shape}\n• Columnas: {ps_2020[x].columns}\n')
for x in dims:
    print(f'• DataFrame: {x}\n• Shape: {dims[x].shape}\n• Columnas: {dims[x].columns}\n')

• DataFrame: 2020-04-26
• Shape: (478909, 3)
• Columnas: Index(['precio', 'sucursal_id', 'producto_id'], dtype='object')

• DataFrame: 2020-04-19
• Shape: (458543, 3)
• Columnas: Index(['precio', 'sucursal_id', 'producto_id'], dtype='object')

• DataFrame: 2020-04-13
• Shape: (472166, 3)
• Columnas: Index(['precio', 'producto_id', 'sucursal_id'], dtype='object')

• DataFrame: 2020-05-03
• Shape: (397734, 3)
• Columnas: Index(['precio', 'producto_id', 'sucursal_id'], dtype='object')

• DataFrame: 2020-05-18
• Shape: (415293, 3)
• Columnas: Index(['precio', 'producto_id', 'sucursal_id'], dtype='object')

• DataFrame: producto
• Shape: (72038, 7)
• Columnas: Index(['id', 'marca', 'nombre', 'presentacion', 'categoria1', 'categoria2',
       'categoria3'],
      dtype='object')

• DataFrame: sucursal
• Shape: (2333, 12)
• Columnas: Index(['id', 'comercioId', 'banderaId', 'banderaDescripcion',
       'comercioRazonSocial', 'provincia', 'localidad', 'direccion', 'lat',
       'lng', 'sucursal

## Proceso de limpieza de datos
Una vez creadas las tablas podemos empezar el trabajo de limpieza de los datos, comenzando por buscar los registros duplicados y valores faltantes.

El propósito de esta sección es:
1) Dejar todas las columnas de las tablas de 'precios' con el mismo tipo de dato y mismo formato de registros en las diferentes tablas para poder concatenarlas después y subirlas a una base de datos SQL.
2) Asegurarnos de que todos los valores de las columnas 'producto_id' y 'sucursal_id' puedan relacionarse a algún valor de las columans 'id' en las dimensiones 'producto' y 'sucursal'

In [6]:
# Organizamos las columnas de los dataframes para que coincidan entre sí y facilitar la visualización
# Además agragamos la columna 'fecha' que nos permitirá distinguir a qué dataset pertenece cada registro una vez sean concatenados
cols = ['precio', 'producto_id', 'sucursal_id']
for x in ps_2020:
    ps_2020[x] = ps_2020[x][cols]
    ps_2020[x]['fecha'] = datetime.strptime(x, '%Y-%m-%d')
    print(f'----------------{x}----------------\n{ps_2020[x].columns}')

----------------2020-04-26----------------
Index(['precio', 'producto_id', 'sucursal_id', 'fecha'], dtype='object')
----------------2020-04-19----------------
Index(['precio', 'producto_id', 'sucursal_id', 'fecha'], dtype='object')
----------------2020-04-13----------------
Index(['precio', 'producto_id', 'sucursal_id', 'fecha'], dtype='object')
----------------2020-05-03----------------
Index(['precio', 'producto_id', 'sucursal_id', 'fecha'], dtype='object')
----------------2020-05-18----------------
Index(['precio', 'producto_id', 'sucursal_id', 'fecha'], dtype='object')


In [7]:
# Usamos un ciclo para obtener la información de todos los DataFrames
for x in ps_2020:
    semana = ps_2020[x]
    print(f'\n-----DataFrame: {x}-----\n\n• Valores nulos:\n{semana.isnull().sum()}\n\n• Registros duplicados:\n{semana.duplicated().value_counts()}\n')
for x in dims:
    dim = dims[x]
    print(f'\n-----DataFrame: {x}-----\n\n• Valores nulos:\n{dim.isnull().sum()}\n\n• Registros duplicados:\n{dim.duplicated().value_counts()}\n')


-----DataFrame: 2020-04-26-----

• Valores nulos:
precio          1736
producto_id    13519
sucursal_id        0
fecha              0
dtype: int64

• Registros duplicados:
False    474692
True       4217
dtype: int64


-----DataFrame: 2020-04-19-----

• Valores nulos:
precio         1807
producto_id       0
sucursal_id       0
fecha             0
dtype: int64

• Registros duplicados:
False    458543
dtype: int64


-----DataFrame: 2020-04-13-----

• Valores nulos:
precio         13
producto_id    15
sucursal_id    15
fecha           0
dtype: int64

• Registros duplicados:
False    472134
True         32
dtype: int64


-----DataFrame: 2020-05-03-----

• Valores nulos:
precio         0
producto_id    0
sucursal_id    0
fecha          0
dtype: int64

• Registros duplicados:
False    397734
dtype: int64


-----DataFrame: 2020-05-18-----

• Valores nulos:
precio         1960
producto_id       6
sucursal_id       6
fecha             0
dtype: int64

• Registros duplicados:
False    415104
Tru

In [8]:
# Habiendo detectado registros duplicados en los datasets de 'ps_2020' procedemos a eliminarlos
for x in ps_2020:
    ps_2020[x].drop_duplicates(inplace=True)

In [9]:
# Ahora podemos generar una visualización preliminar de las tablas para continuar con el proceso de limpieza y poder lidiar con los valores nulos
for x in ps_2020:
    print(f'-----DataFrame: {x}-----\n{ps_2020[x].head(15)}')
for x in dims:
    print(f'-----DataFrame: {x}-----\n{dims[x].head(15)}')

-----DataFrame: 2020-04-26-----
     precio  producto_id sucursal_id      fecha
0     399.0       2288.0     2-1-092 2020-04-26
1     299.0       2288.0     2-1-206 2020-04-26
2     399.0       2288.0     2-2-241 2020-04-26
3   49999.0     205870.0     9-1-430 2020-04-26
4   53999.0     205870.0       9-2-4 2020-04-26
5   53999.0     205870.0    9-3-5218 2020-04-26
6   58999.0     205894.0     9-1-430 2020-04-26
7   18999.0     205955.0     9-1-430 2020-04-26
8   10499.0     205979.0     9-1-430 2020-04-26
9    2290.0     206020.0     9-1-430 2020-04-26
10  27999.0     206044.0     9-1-430 2020-04-26
11   2190.0     206044.0     9-1-691 2020-04-26
12  22999.0     206051.0     9-1-430 2020-04-26
13  45999.0     206105.0     9-1-430 2020-04-26
14  38999.0     206136.0     9-1-430 2020-04-26
-----DataFrame: 2020-04-19-----
    precio producto_id          sucursal_id      fecha
0    29.90        2288              2-1-184 2020-04-19
1    39.90        2288              2-1-206 2020-04-19
2  

In [10]:
# Revisamos también la información de los diferentes datasets para tener presente el tipo de dato de cada columna
for x in ps_2020:
    print(ps_2020[x].info())
for x in dims:
    print(dims[x].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474692 entries, 0 to 478908
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   precio       473053 non-null  float64       
 1   producto_id  465390 non-null  float64       
 2   sucursal_id  474692 non-null  object        
 3   fecha        474692 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 18.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 458543 entries, 0 to 458542
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   precio       456736 non-null  float64       
 1   producto_id  458543 non-null  object        
 2   sucursal_id  458543 non-null  object        
 3   fecha        458543 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 17.5+ MB
None
<class 'pandas.core.frame.

### • Primeros problemas con el formato de los datos:
Con la anterior visualización preliminar y la información de los DataFrames se pueden ver varias cuestiones a tener en cuenta y que se abordarán a continuación:
1) En uno de los DataFrames de precios (2020-05-03), el tipo de data de la columna 'precio' no es *float64* como en los demás.
2) El formato de la columna 'producto_id' es diferente en varios DataFrames.
3) Algunos valores de la columna 'sucursal_id' de al menos la tabla '2020-04-19' son interpretados en como de tipo 'datetime'.
4) 

### 1) El tipo de data de la columna 'precio' no es *float64* en todos los DataFrames

In [11]:
# Ahora exploramos un poco los datos de '2020-05-03.precio', columna que no era del tipo 'float64'
# Obtenemos 1) el tipo de datos que encontramos en cada columna
#           2) el valor de los datos tipo 'str'
#           3) la cantidad de ceros en esta columna
precio_dtype = set()
strings = set()
zeros = 0
for x in ps_2020['2020-05-03'].precio:
    precio_dtype.add(str(type(x)))
    if type(x) == str:
        #print(x)
        strings.add(x)
    elif x == 0:
        zeros += 1
print(f'• Tipos de datos que encontramos en la columna precios del DataFrame 2020-05-03:\n{precio_dtype}\n')
print(f'• Valores que encontramos para los strings:\n{strings}\n')
print(f'• Apariciones de 0 en la columna:\n{zeros}')

• Tipos de datos que encontramos en la columna precios del DataFrame 2020-05-03:
{"<class 'int'>", "<class 'float'>", "<class 'str'>"}

• Valores que encontramos para los strings:
{''}

• Apariciones de 0 en la columna:
0


##### • Ya que detectamos que no hay registros en la columna con valor de '0', éste es un buen candidato para remplazar por ahora los valores faltantes de 'precio'

In [12]:
# Procedemos a detectar si en los demás DataFrames hay valores de '0' en la columna precio
has_zeros = {}
for x in ps_2020:
    has_zeros[x] = (0 in ps_2020[x].precio.unique())
for x in has_zeros:
    print(f"DataFrame: {x}\nEstá 0 en la columna 'precio': {has_zeros[x]}\n")

DataFrame: 2020-04-26
Está 0 en la columna 'precio': False

DataFrame: 2020-04-19
Está 0 en la columna 'precio': False

DataFrame: 2020-04-13
Está 0 en la columna 'precio': False

DataFrame: 2020-05-03
Está 0 en la columna 'precio': False

DataFrame: 2020-05-18
Está 0 en la columna 'precio': False



In [13]:
# Dado que ningún DataFrame tiene el dato '0' en la columna 'precio', procedemos a remplazar los valores faltantes con '0'
for x in ps_2020:
    ps_2020[x]['precio_ok'] = ps_2020[x]['precio'].replace('', np.nan).fillna(0)

In [14]:
# Volvemos a explorar los datos de '2020-05-03.precio', columna que no era del tipo 'float64'
precio_dtype = set()
zeros = 0
for x in ps_2020['2020-05-03'].precio_ok:
    precio_dtype.add(str(type(x)))
    if x == 0:
        zeros += 1
print(f'• Tipos de datos que encontramos en la columna precios_ok del DataFrame 2020-05-03:\n{precio_dtype}\n')
print(f'• Apariciones de 0 en la columna:\n{zeros}')

• Tipos de datos que encontramos en la columna precios_ok del DataFrame 2020-05-03:
{"<class 'float'>"}

• Apariciones de 0 en la columna:
2124


Con el anterior test podemos corroborar que la nueva columna ('precio_ok') ya no tiene valores del tipo *str* así que esta es la columna que usaremos en el dataset final.

### 2) Unificar el formato de las columnas 'producto_id', dejándolo como un *str*

In [15]:
# Primero nos cercioramos del tipo de dato que hay en cada una de las columnas 'producto_id'
producto_id_dtypes = {}
for x in ps_2020:
    producto_id_dtypes[x] = set()
    for y in ps_2020[x].producto_id:
        producto_id_dtypes[x].add(type(y))
for x in producto_id_dtypes:
    print(f"• DataFrame {x}:\n  Tipos de dato en la columna 'producto_id': {producto_id_dtypes[x]}")


• DataFrame 2020-04-26:
  Tipos de dato en la columna 'producto_id': {<class 'float'>}
• DataFrame 2020-04-19:
  Tipos de dato en la columna 'producto_id': {<class 'str'>, <class 'int'>}
• DataFrame 2020-04-13:
  Tipos de dato en la columna 'producto_id': {<class 'str'>, <class 'float'>}
• DataFrame 2020-05-03:
  Tipos de dato en la columna 'producto_id': {<class 'str'>, <class 'int'>}
• DataFrame 2020-05-18:
  Tipos de dato en la columna 'producto_id': {<class 'str'>, <class 'float'>}


In [16]:
# Revisamos la cantidad de valores de tipo 'float' en las tablas 2020-04-26, 2020-04-13 y 2020-05-18
# Además revisamos si encontramos '0' entre estos valores
valores_float = {}
pid_zeros = 0
for x in ps_2020:
    valores_float[x] = set()
    for y in ps_2020[x].producto_id:
        if type(y) == float:
            valores_float[x].add(y)
        if y in [0,'0']:
            pid_zeros += 1
for x in valores_float:
    print(f"\n• DataFrame {x}:\n  Cantidad de valores 'float' en la columna 'producto_id': {len(valores_float[x])}")
    if len(valores_float[x]) ==1:
        print(f"    El único valor float encontrado fue: {valores_float[x]}")
print(f"\n• Cantidad de ceros en las columnas 'producto_id': {pid_zeros}")


• DataFrame 2020-04-26:
  Cantidad de valores 'float' en la columna 'producto_id': 63925

• DataFrame 2020-04-19:
  Cantidad de valores 'float' en la columna 'producto_id': 0

• DataFrame 2020-04-13:
  Cantidad de valores 'float' en la columna 'producto_id': 1
    El único valor float encontrado fue: {nan}

• DataFrame 2020-05-03:
  Cantidad de valores 'float' en la columna 'producto_id': 0

• DataFrame 2020-05-18:
  Cantidad de valores 'float' en la columna 'producto_id': 1
    El único valor float encontrado fue: {nan}

• Cantidad de ceros en las columnas 'producto_id': 0


Del anterior chequeo podemos deducir que sólo el DataFrame 2020-04-26 tiene valores 'float' distintos de 'nan', además de que ninguno de los DataFrames tiene el valor '0' en la columna 'producto_id'. 

Por lo anterior, procederemos a remplazar los valores faltantes con 0 y, posteriormente convertir los valores 'float' en 'int' (para quitar el decimal) y luego en 'str' (para homogeneizar las claves).

In [17]:
# Remplazamos los valores faltantes con 0
for x in ps_2020:
    ps_2020[x]['producto_id2'] = (ps_2020[x]['producto_id'].replace('', np.nan).fillna(0))

'''
# Revisamos si todavía hay valores nulos
for x in ps_2020:
    print(f'\n• DataFrame: {x}')
    print(ps_2020[x].isnull().sum())
'''

"\n# Revisamos si todavía hay valores nulos\nfor x in ps_2020:\n    print(f'\n• DataFrame: {x}')\n    print(ps_2020[x].isnull().sum())\n"

In [18]:
# Creamos una función para cambiar los valores 'float' e 'int' a 'str'
def pid_num2str(val):
    if type(val) == float:
        return str(int(val))
    elif type(val) == int:
        return str(val)
    else:
        return val
    
# Aplicamos la función a la columna sin valores nulos (producto_id2) y guardamos el resultado en otra columna (producto_id3)
for x in ps_2020:
    ps_2020[x]['producto_id3'] = ps_2020[x]['producto_id2'].apply(pid_num2str)

In [19]:
# Ahora revisamos el tipo de dato que hay en cada una de las columnas 'producto_id3'
sucursal_id_dtypes = {}
for x in ps_2020:
    sucursal_id_dtypes[x] = set()
    for y in ps_2020[x].producto_id3:
        sucursal_id_dtypes[x].add(type(y))
for x in sucursal_id_dtypes:
    print(f"• DataFrame {x}:\n  Tipos de dato en la columna 'producto_id3': {sucursal_id_dtypes[x]}")

• DataFrame 2020-04-26:
  Tipos de dato en la columna 'producto_id3': {<class 'str'>}
• DataFrame 2020-04-19:
  Tipos de dato en la columna 'producto_id3': {<class 'str'>}
• DataFrame 2020-04-13:
  Tipos de dato en la columna 'producto_id3': {<class 'str'>}
• DataFrame 2020-05-03:
  Tipos de dato en la columna 'producto_id3': {<class 'str'>}
• DataFrame 2020-05-18:
  Tipos de dato en la columna 'producto_id3': {<class 'str'>}


##### Ya que tenemos el 'producto_id' en todas las tablas de 'precios' en formato 'str', nos aseguraremos ahora de que estos valores correspondan a un valor en la tabla 'producto', recordando que los datos faltantes fueron remplazados por 0.

In [20]:
# Revisamos el formato del 'id' en la tabla producto a la que las columnas 'precio.producto_id' harán referencia
# Para esto vemos primero los tipos de dato almacenados en esta columna
pid_dtypes = set()
for x in dims['producto'].id:
    pid_dtypes.add(type(x))
for x in pid_dtypes:
    print(x)

<class 'str'>


In [21]:
# Al confirmar que todos los valores de la columna 'id' en la tabla 'producto' son str, obtenemos la longitud de estos valores
pid_lengths = set()

for x in dims['producto'].id.unique():
    pid_lengths.add(len(x))

print(f"En la columna 'id' de la tabla 'producto' encontramos valores de las siguientes longitudes: {pid_lengths}")

En la columna 'id' de la tabla 'producto' encontramos valores de las siguientes longitudes: {17, 18, 13}


In [22]:
# Revisamos ahora en qué consisten las diferencias de los códigos de distintas longitudes
pid_lengths_examples = {}
for x in pid_lengths:
    pid_lengths_examples[str(x)] = []
for x in dims['producto'].id:
    len_x = str(len(x))
    pid_lengths_examples[len_x].append(x)

print("\n• Algunos ejemplos de los valores en la columna 'id' de la tabla 'producto' con diferentes longitudes  (longitud: ejemplos):\n")
for x in pid_lengths_examples:
    print(f'{x}: {sample(pid_lengths_examples[x],5)}')


• Algunos ejemplos de los valores en la columna 'id' de la tabla 'producto' con diferentes longitudes  (longitud: ejemplos):

17: ['9-1-0000000992404', '9-1-0000000969581', '7-1-0024006900000', '9-1-0000000256124', '9-1-0000000206723']
18: ['16-1-0000000263091', '46-1-0000000083074', '10-2-2308052000008', '10-2-2307317000005', '65-1-0000000014788']
13: ['8480017419538', '7798126290315', '7797470196342', '7798133610557', '7790520009357']


Se puede ver que los códigos de 'id' en la tabla 'producto' con más de 13 caracteres se diferencian de aquellos con 13 caracteres porque tienen un prefijo compuesto de la siguiente manera:

(Número de uno o dos dígitos)+(guión)+(Número de un dígito)+(guión)

Dado que hay diferentes longitudes en los códigos de la columna 'id' de la tabla 'producto', revisaremos ahora si:
1) Estos códigos son únicos por registro
2) Los códigos siguen siendo únicos si los reducimos todos a tener sólo 13 caracteres (los últimos 13, para evitar la parte que contiene guiones)

In [23]:
# 1) Se verifica que los ids originales sean únicos por registro

tot_reg_producto = len(dims['producto'])
id_uniq_producto = dims['producto'].id.unique().shape[0]
print(f"Cantidad de registros en la tabla 'producto': {tot_reg_producto}")
print(f"Cantidad de valores únicos en la columna 'id' original: {id_uniq_producto}")
print(f"\n• Los valores originales de la columna 'id' son únicos por registro: {tot_reg_producto==id_uniq_producto}\n")


Cantidad de registros en la tabla 'producto': 72038
Cantidad de valores únicos en la columna 'id' original: 72038

• Los valores originales de la columna 'id' son únicos por registro: True



In [24]:
# Creamos un nuevo DataFrame a partir de 'producto' y remplazamos la columna 'id' por su versión reducida (pid2)
pid2 = []
for x in dims['producto'].id:
    if len(x)>13:
        y = x[-13:]
        pid2.append(y)
    else:
        pid2.append(x)
pid2_array = pd.Series(pid2)

# Creamos un DataFrame nuevo con los ids reducidos y dropeando los duplicados
dims['producto2'] = dims['producto'].drop('id', axis=1)

dims['producto2']['id'] = pid2_array

dims['producto2'].drop_duplicates(inplace=True)

print(dims['producto2'].columns)

print(f"\n-----DataFrame: 'producto2'-----\n\n• Valores nulos:\n{dims['producto2'].isnull().sum()}\n\n• Registros duplicados:\n{dims['producto2'].duplicated().value_counts()}\n")

Index(['marca', 'nombre', 'presentacion', 'categoria1', 'categoria2',
       'categoria3', 'id'],
      dtype='object')

-----DataFrame: 'producto2'-----

• Valores nulos:
marca               2
nombre              2
presentacion        2
categoria1      68194
categoria2      68194
categoria3      68194
id                  0
dtype: int64

• Registros duplicados:
False    68198
dtype: int64



In [25]:
# 2) Verificamos si el nuevo DataFrame con ids reducidos ('producto2') sigue teniendo un id único para cada registro
tot_reg_producto2 = len(dims['producto2'])
id_uniq_producto2 = dims['producto2'].id.unique().shape[0]
print(f"Cantidad de registros en la tabla 'producto2': {tot_reg_producto2}")
print(f"Cantidad de valores únicos en la columna 'id' reducida: {id_uniq_producto2}")
print(f"\n• Los valores reducidos de la columna 'id' son únicos por registro: {tot_reg_producto2==id_uniq_producto2}\n")

Cantidad de registros en la tabla 'producto2': 68198
Cantidad de valores únicos en la columna 'id' reducida: 67943

• Los valores reducidos de la columna 'id' son únicos por registro: False



#### Con el código de las últimas 3 celdas se logró verificar que es necesario conservar los ids originales de la tabla 'producto' puesto que, si tomamos sólo los últimos 13 caracteres de cada código, entonces tendremos códigos repetidos para diferentes productos.

Dado que no se puede reducir la longitud de los códigos de 'id' largos sin comprometer la unicidad de cada código para los registros de la tabla 'producto', procederemos a modificar las columnas 'producto_id' de las tablas 'precios' para que se ajusten a alguno de los valores de 13, 17 o 18 dígitos de la columna 'id' de la tabla 'producto'.

In [26]:
# Empezamos por revisar las diferentes longitudes de los valores de las columnas 'producto_id' en las tablas de 'precios' 
# Posteriormente visualizamos los valores más excepcionales (1 y 4)
ps_pid_lengths = {}
id_1_car = []
id_4_car = []
for x in ps_2020:
    ps_pid_lengths[x] = set()
    for y in ps_2020[x]['producto_id3'].unique():
        ps_pid_lengths[x].add(len(str(y)))
        if len(str(y)) == 1:
            id_1_car.append([x,y])
        elif len(str(y)) == 4:
            id_4_car.append([x,y])

print('\n• Cantidades de caracteres en los distintos valores de las columnas producto_id3:\n')
for x in ps_pid_lengths:
    print(f'\tDataFrame{x}: {ps_pid_lengths[x]}\n')

print('• Casos más excepcionales (con valores de 1 o 4 caracteres):\n')
print(f'\tValores de 1 caracter por DataFrame (df, valor): {id_1_car}\n')
print(f'\tValores de 4 caracteres por DataFrame (df, valor): {id_4_car}')


• Cantidades de caracteres en los distintos valores de las columnas producto_id3:

	DataFrame2020-04-26: {1, 4, 6, 7, 8, 9, 10, 11, 12, 13}

	DataFrame2020-04-19: {4, 6, 7, 8, 9, 10, 11, 12, 13, 17, 18}

	DataFrame2020-04-13: {17, 18, 13, 1}

	DataFrame2020-05-03: {17, 18, 13}

	DataFrame2020-05-18: {17, 18, 13, 1}

• Casos más excepcionales (con valores de 1 o 4 caracteres):

	Valores de 1 caracter por DataFrame (df, valor): [['2020-04-26', '0'], ['2020-04-13', '0'], ['2020-05-18', '0']]

	Valores de 4 caracteres por DataFrame (df, valor): [['2020-04-26', '2288'], ['2020-04-19', '2288']]


Se puede ver que el caso más extremo corresponde al de los valores nulos, que fueron remplazados por 0.

Mientras que el siguiente caso de valores con pocos caracteres es '2288', valor que s epodría corresponder con el id '0000000002288' de la tabla 'producto'.

Por esta razón, procederemos a llenar los valores con menos de 13 caracteres de las columnas 'producto_id3' en las tablas 'precios' con 0's a la izquierda hasta tener un valor con 13 caracteres.

In [27]:
# Primero vemos la cantidad de registros con menos de 13 caracteres en cada uno de los DataFrames de 'precios'
pid_short_codes = {}
for x in ps_2020:
    pid_short_codes[x] = []
    for y in ps_2020[x].producto_id3:
        if len(y)<13:
            pid_short_codes[x].append(x)
print("Cantidad de códigos en las columnas 'producto_id3' con menos de 13 caracteres por DataFrame:")
for x in pid_short_codes:
    print(f'\n• DataFrame {x}:{len(pid_short_codes[x])}')

Cantidad de códigos en las columnas 'producto_id3' con menos de 13 caracteres por DataFrame:

• DataFrame 2020-04-26:20736

• DataFrame 2020-04-19:11125

• DataFrame 2020-04-13:3

• DataFrame 2020-05-03:0

• DataFrame 2020-05-18:3


In [28]:
# Creamos una función que nos regrese un valor de 13 caracteres con 0 a la izquierda si el valor tiene argumento pasado tiene menos de 13
def fill_id(str):
    if len(str)<13:
        return str.zfill(13)
    else:
        return str

In [29]:
# Aplicamos la función a las columnas 'producto_id3' de los DataFrames de 'precios'
for x in ps_2020:
    ps_2020[x]['producto_id4'] = ps_2020[x]['producto_id3'].apply(fill_id)

In [30]:
# Revisamos las longitudes de los valores de la columna 'producto_id4'
pid4_lengths = set()
for x in ps_2020:
    for y in ps_2020[x].producto_id4:
        pid4_lengths.add(len(y))
print(pid4_lengths)

{17, 18, 13}


##### • Una vez transformados los datos de la columna 'producto_id' de los DataFrames de 'precios' a su forma final en la columna 'producto_id4', procedemos a revisar que todos estos sí hagan referencia a algún valor de la columna 'id' en la tabla 'producto'.

In [31]:
# Creamos una lista (pid) con los valores de la columna 'id' de la tabla producto
pid = []

for x in dims['producto'].id.unique():
    pid.append(x)
print(len(pid))
print(dims['producto'].shape[0])

72038
72038


In [32]:
# El siguiente código revisa, para todas los DataFrames de precios, si los valores de 'producto_id4' se encuentran en la lista 'pid' recién creada.
# La ejecución puede llegar a tardar varios minutos así que el código está comentado para que no se ejecute automáticamente al correr todas las celdas.
'''
pid_checklist = {}
pid_problems_found = {}
for x in ps_2020:
    len_x = len(ps_2020[x].producto_id4.unique())
    print(f'Checking DataFrame {x}... {len_x} values to check..')
    pid_checklist[x] = False
    pid_problems_found[x] = []
    for y in ps_2020[x].producto_id4.unique():
        if (y != '0000000000000'):
            if y not in pid:
                pid_checklist[x] = True
                pid_problems_found[x].append(y)
                #print(f'Problem found: (df:{x}) (value:{y}) (dtype:{type(y)})')

print('\n\nRESULTS:\n')
for x in pid_checklist:
    print(f"• DataFrame {x}:\n\tFound problems: {pid_checklist[x]} (Amount: {len(pid_problems_found[x])})")
'''

''' Output (8m 30.2s):
Checking 2020-04-26... 54624 values to check..
Checking 2020-04-19... 62393 values to check..
Checking 2020-04-13... 66532 values to check..
Checking 2020-05-03... 61505 values to check..
Checking 2020-05-18... 60745 values to check..


RESULTS:

• DataFrame 2020-04-26:
	Found problems: False (Amount: 0)
• DataFrame 2020-04-19:
	Found problems: True (Amount: 7)
• DataFrame 2020-04-13:
	Found problems: False (Amount: 0)
• DataFrame 2020-05-03:
	Found problems: False (Amount: 0)
• DataFrame 2020-05-18:
	Found problems: False (Amount: 0)
'''

' Output (8m 30.2s):\nChecking 2020-04-26... 54624 values to check..\nChecking 2020-04-19... 62393 values to check..\nChecking 2020-04-13... 66532 values to check..\nChecking 2020-05-03... 61505 values to check..\nChecking 2020-05-18... 60745 values to check..\n\n\nRESULTS:\n\n• DataFrame 2020-04-26:\n\tFound problems: False (Amount: 0)\n• DataFrame 2020-04-19:\n\tFound problems: True (Amount: 7)\n• DataFrame 2020-04-13:\n\tFound problems: False (Amount: 0)\n• DataFrame 2020-05-03:\n\tFound problems: False (Amount: 0)\n• DataFrame 2020-05-18:\n\tFound problems: False (Amount: 0)\n'

In [33]:
'''
for x in pid_problems_found:
    print(f'{x}: {pid_problems_found[x]}')
'''

''' Output:
2020-04-26: []
2020-04-19: ['10-1-2303809000003', '10-2-2304238000008', '2630399000008', '2920433200007', '7790513005687', '7798037563669', '7798037563683']
2020-04-13: []
2020-05-03: []
2020-05-18: []
'''

" Output:\n2020-04-26: []\n2020-04-19: ['10-1-2303809000003', '10-2-2304238000008', '2630399000008', '2920433200007', '7790513005687', '7798037563669', '7798037563683']\n2020-04-13: []\n2020-05-03: []\n2020-05-18: []\n"

#### • Con las últimas tres celdas pudimos corroborar que no todos los valores de 'producto_id4' corresponden a un id de la tabla 'producto'.
Un problema del que habrá que informar al cliente. Sin embargo, al ser pocos (7) los valores de 'producto_id' problemáticos y en una sola tabla, se optará por remplazarlos por el un valor alternativo que usamos en donde los datos faltaban '0000000000000' : '1111111111111'.
Por lo pronto, todos los 'producto_id4' son  del tipo 'str' y de estructura más o menos uniforme (salvo por la variacion en las longitudes) así que se subirán así a la base de datos, una vez hecho el remplazo que se acaba de mencionar.

In [34]:
# Confirmamos si el código a usar para notificar de este error no está ya en los valores 'id' del DataFrame 'producto'
print(('1111111111111' in dims['producto'].id))

False


In [36]:
# Definimos los registros a remplazar (pid_a_remplazar) y vemos cual es el total de registros en las columnas 'producto_id4' a remplazar
pid_a_remplazar = ['10-1-2303809000003', '10-2-2304238000008', '2630399000008', '2920433200007', '7790513005687', '7798037563669', '7798037563683']
# Aquí podemos ver que el total de registros remplazados fueron 7
pid_ar_count = 0
for x in ps_2020:
    for y in ps_2020[x].producto_id4:
        if y in pid_a_remplazar:
            pid_ar_count += 1
tot_reg = 0
for x in ps_2020:
    tot_reg += ps_2020[x].shape[0]
print(f'El total de productos a remplazar en los {len(ps_2020.keys())} DataFrames es de {pid_ar_count}, de un total de {tot_reg} ({round(100*(pid_ar_count/tot_reg),5)}%).')

El total de productos a remplazar en los 5 DataFrames es de 7, de un total de 2218207 (0.00032%).


Al ver que la cantidad de registros a modificar es nimia, llevamos a cabo el remplazo.

In [37]:

# Creamos una función que cambie estos registros por el código elegido
def nxcode2err111(registro):
    if registro in pid_a_remplazar:
        return '1111111111111'
    else:
        return registro
# Aplicamos la función y creamos la columna 'producto_id_ok':
for x in ps_2020:
    ps_2020[x]['producto_id_ok'] = ps_2020[x]['producto_id4'].apply(nxcode2err111)

In [38]:
# Volvemos a hacer la verificación de las referencias de producto_id_ok' a 'producto.id'
'''
pid_checklist = {}
pid_problems_found = {}
for x in ps_2020:
    len_x = len(ps_2020[x].producto_id_ok.unique())
    #print(f'Checking DataFrame {x}... {len_x} values to check..')
    pid_checklist[x] = False
    pid_problems_found[x] = []
    for y in ps_2020[x].producto_id_ok.unique():
        if (y != '0000000000000') and (y != '1111111111111'):
            if y not in pid:
                pid_checklist[x] = True
                pid_problems_found[x].append(y)
                #print(f'Problem found: (df:{x}) (value:{y}) (dtype:{type(y)})')

print('\n\nRESULTS:\n')
for x in pid_checklist:
    print(f"• DataFrame {x}:\n\tFound problems: {pid_checklist[x]} (Amount: {len(pid_problems_found[x])})")
'''

''' Output (9m 37.8s):
RESULTS:

• DataFrame 2020-04-26:
	Found problems: False (Amount: 0)
• DataFrame 2020-04-19:
	Found problems: False (Amount: 0)
• DataFrame 2020-04-13:
	Found problems: False (Amount: 0)
• DataFrame 2020-05-03:
	Found problems: False (Amount: 0)
• DataFrame 2020-05-18:
	Found problems: False (Amount: 0)
' Output (8m 4.8s):\n\n'
'''

" Output (9m 37.8s):\nRESULTS:\n\n• DataFrame 2020-04-26:\n\tFound problems: False (Amount: 0)\n• DataFrame 2020-04-19:\n\tFound problems: False (Amount: 0)\n• DataFrame 2020-04-13:\n\tFound problems: False (Amount: 0)\n• DataFrame 2020-05-03:\n\tFound problems: False (Amount: 0)\n• DataFrame 2020-05-18:\n\tFound problems: False (Amount: 0)\n' Output (8m 4.8s):\n\n'\n"

### 3) Corregir los valores de las columnas 'sucursal_id' que se hayan remplazado por valores tipo *datetime*

In [39]:
# Primero nos cercioramos del tipo de dato que hay en cada una de las columnas 'sucursal_id'
sucursal_id_dtypes = {}
for x in ps_2020:
    sucursal_id_dtypes[x] = set()
    for y in ps_2020[x].sucursal_id:
        sucursal_id_dtypes[x].add(type(y))
for x in sucursal_id_dtypes:
    print(f"• DataFrame {x}:\nTipos de datos encontrados en la columna 'sucursal_id': {sucursal_id_dtypes[x]}\n")

• DataFrame 2020-04-26:
Tipos de datos encontrados en la columna 'sucursal_id': {<class 'str'>}

• DataFrame 2020-04-19:
Tipos de datos encontrados en la columna 'sucursal_id': {<class 'datetime.datetime'>, <class 'str'>}

• DataFrame 2020-04-13:
Tipos de datos encontrados en la columna 'sucursal_id': {<class 'str'>, <class 'float'>}

• DataFrame 2020-05-03:
Tipos de datos encontrados en la columna 'sucursal_id': {<class 'str'>}

• DataFrame 2020-05-18:
Tipos de datos encontrados en la columna 'sucursal_id': {<class 'str'>, <class 'float'>}



Vemos que además de los valores datetime (que sólo están en el DataFrame 2020-04-19) también hay valores 'float' en los DataFrames 2020-04-13 y 2020-05-18.

In [40]:
# Revisamos qué tantos valores encontramos para cada tipo y si son pocos los visualizamos
count = 0
dt_count = 0
dt_values = {}
fl_count = 0
fl_values = {}

for x in ps_2020:
    dt_values[x] = []
    fl_values[x] = []
    for y in ps_2020[x].sucursal_id:
        if type(y) != str:
            count += 1
            if type(y) == datetime:
                dt_count += 1
                dt_values[x].append(y)
            elif type(y) == float:
                fl_count += 1
                fl_values[x].append(y)
print(f"\nValores que no son de tipo 'str': {count}\n")
print(f"• Tipo datetime ({len(dt_values)}): {dt_count}")
for x in dt_values:
    print(f"\t{x}:{len(dt_values[x])}")
print(f"• Tipo float: {fl_count}")
for x in fl_values:
    print(f"\t{x}:{len(fl_values[x])} -- values: {fl_values[x]}")


Valores que no son de tipo 'str': 140449

• Tipo datetime (5): 140443
	2020-04-26:0
	2020-04-19:140443
	2020-04-13:0
	2020-05-03:0
	2020-05-18:0
• Tipo float: 6
	2020-04-26:0 -- values: []
	2020-04-19:0 -- values: []
	2020-04-13:3 -- values: [nan, nan, nan]
	2020-05-03:0 -- values: []
	2020-05-18:3 -- values: [nan, nan, nan]


Con el anterior testeo podemos ver que los valores de tipo 'float' que se registran en los DataFrames 2020-04-13 y 2020-05-18 son valores faltantes. En lo siguiente serán remplazados por el str '0' para que todas las columnas sean de tipo str.

In [41]:
# Creamos una función que se pueda aplicar a la columna 'sucursal_id' y nos regrese cada valor como un str en el formato deseado
def sucursal_id_2str(registro):
    if type(registro) == datetime:
        spl = registro.strftime('%d-%m-%Y').split('-')
        #print(f'{x}:{spl}')
        if spl[0][0] == '0':
            spl[0] = spl[0][1]
        if spl[1][0] == '0':
            spl[1] = spl[1][1]
        return str(f'{spl[0]}-{spl[1]}-{spl[2]}')
    elif type(registro) == float:
        return '0'
    else:
        return str(registro)

In [42]:
# Aplicamos la función a las columnas de 'sucursal_id' y guardamos los valores en una nueva columna 'sucursal_id2'
for x in ps_2020:
    ps_2020[x]['sucursal_id2'] = ps_2020[x].sucursal_id.apply(sucursal_id_2str)

In [43]:
# Revisamos los tipos de datos hallados en las columnas recien creadas (sucursal_id2)
sucursal_id2_dtypes = {}
for x in ps_2020:
    sucursal_id2_dtypes[x] = set()
    for y in ps_2020[x].sucursal_id2:
        sucursal_id2_dtypes[x].add(type(y))
for x in sucursal_id2_dtypes:
    print(f"• DataFrame {x}:\nTipos de datos encontrados en la columna 'sucursal_id2': {sucursal_id2_dtypes[x]}\n")

• DataFrame 2020-04-26:
Tipos de datos encontrados en la columna 'sucursal_id2': {<class 'str'>}

• DataFrame 2020-04-19:
Tipos de datos encontrados en la columna 'sucursal_id2': {<class 'str'>}

• DataFrame 2020-04-13:
Tipos de datos encontrados en la columna 'sucursal_id2': {<class 'str'>}

• DataFrame 2020-05-03:
Tipos de datos encontrados en la columna 'sucursal_id2': {<class 'str'>}

• DataFrame 2020-05-18:
Tipos de datos encontrados en la columna 'sucursal_id2': {<class 'str'>}



Con el anterior chequeo podemos corroborar que la nueva columna creada ya no tiene valores del tipo 'datetime' ni 'float'.

Ahora revisamos si los valores de las columnas 'sucursal_id' distintos de '0' hacen referencia a algún valor de la columna 'id' de la tabla 'sucursal'. Para esto empezamos por revisar si las longitudes de los strings se corresponden.

In [44]:
# Revisamos el formato del 'id' en la tabla sucursal a la que las columnas 'precio.sucursal_id' harán referencia
# Para esto vemos primero los tipos de dato almacenados en esta columna
sid_dtypes = set()
for x in dims['sucursal'].id:
    sid_dtypes.add(type(x))
for x in sid_dtypes:
    print(x)

<class 'str'>


In [45]:
# Al confirmar que todos los valores de la columna 'id' en la tabla 'sucursal' son str, obtenemos la longitud de estos valores
sid_lengths = set()

for x in dims['sucursal'].id.unique():
    sid_lengths.add(len(x))

print(f"En la columna 'id' de la tabla 'sucursal' encontramos valores de las siguientes longitudes: {sid_lengths}")

En la columna 'id' de la tabla 'sucursal' encontramos valores de las siguientes longitudes: {5, 6, 7, 8, 9, 10}


In [46]:
# Revisamos ahora en qué consisten las diferencias de los códigos de distintas longitudes
sid_lengths_examples = {}
for x in sid_lengths:
    sid_lengths_examples[str(x)] = []
for x in dims['sucursal'].id:
    len_x = str(len(x))
    sid_lengths_examples[len_x].append(x)

print("\nAlgunos ejemplos de los valores en la columna 'id' de la tabla 'sucursal' con diferentes longitudes  (longitud: ejemplos):\n")
for x in sid_lengths_examples:
    print(f'{x}: {sample(sid_lengths_examples[x],5)}')


Algunos ejemplos de los valores en la columna 'id' de la tabla 'sucursal' con diferentes longitudes  (longitud: ejemplos):

5: ['6-2-1', '8-1-7', '9-1-8', '8-1-9', '7-1-5']
6: ['6-2-21', '7-1-47', '9-2-15', '18-1-6', '25-1-1']
7: ['12-1-65', '15-1-19', '15-1-89', '9-1-767', '2-1-056']
8: ['15-1-412', '15-1-234', '9-3-5225', '10-3-526', '9-3-5276']
9: ['15-1-1523', '15-1-1505', '15-1-5242', '15-1-1006', '23-1-6205']
10: ['19-1-02697', '19-1-02460', '19-1-03228', '19-1-01317', '19-1-01641']


Se puede ver que para todos las longitudes de los códigos, la estructura de estos consiste en 3 números separados por guiones.

Cabe destacar que en los códigos más largos (de 10 caracteres) el tercer número parece estar precedido por ceros a la izquierda que, de ser ignorados, podrían hacer que el código se igual a uno de 9 u 8 caracteres.

In [47]:
# Verificamos que los valores de 'id' en el DataFrame 'sucursal' sean únicos por registro
tot_reg_sucursal = len(dims['sucursal'])
id_uniq_sucursal = dims['sucursal'].id.unique().shape[0]
print(f"Cantidad de registros en la tabla 'sucursal': {tot_reg_sucursal}")
print(f"Cantidad de valores únicos en la columna 'id' original: {id_uniq_sucursal}")
print(f"\n• Los valores originales de la columna 'id' son únicos por registro: {tot_reg_sucursal==id_uniq_sucursal}\n")

Cantidad de registros en la tabla 'sucursal': 2333
Cantidad de valores únicos en la columna 'id' original: 2333

• Los valores originales de la columna 'id' son únicos por registro: True



In [48]:
# Revisamos si podemos tomar el último de los numeros de la columna 'id' (sid_v3) en el DataFrame 'sucursal' para identificar los registros de manera única
# O bien la combinación del último valor con el primero (sid_v1_v3) o con el segundo (sid_v2_v3)
tot_reg_sucursal = dims['sucursal'].shape[0]
sid_v3 = []
sid_v1_v3 = []
sid_v2_v3 = []
for x in dims['sucursal'].id:
    sid_split = x.split('-')
    sid_v3.append(sid_split[2])
    sid_v1_v3.append(sid_split[0]+sid_split[2])
    sid_v2_v3.append(sid_split[1]+sid_split[2])
sid_v3_Srs = pd.Series(sid_v3)
sid_v1_v3_Srs = pd.Series(sid_v1_v3)
sid_v2_v3_Srs = pd.Series(sid_v2_v3)

print(f"• Total de registros en el DataFrame 'sucursal': {tot_reg_sucursal}")
print(f"• Total de valores únicos en la columna 'id' del DataFrame 'sucursal': {len(dims['sucursal'].id.unique())}")
print(f"\n• Totales de valores únicos en la columna hipotética 'id_v3' (tomando únicamente el último número de 'id') del DataFrame 'sucursal':\n{len(sid_v3_Srs.unique())}")
print(f"\n• Totales de valores únicos en la columna hipotética 'id_v1_v3' (tomando la combinación del primer y el último número de 'id') del DataFrame 'sucursal':\n{len(sid_v1_v3_Srs.unique())}")
print(f"\n• Totales de valores únicos en la columna hipotética 'id_v2_v3' (tomando la combinación del segundo y el último número de 'id') del DataFrame 'sucursal':\n{len(sid_v2_v3_Srs.unique())}")

#print(sid_v3)

• Total de registros en el DataFrame 'sucursal': 2333
• Total de valores únicos en la columna 'id' del DataFrame 'sucursal': 2333

• Totales de valores únicos en la columna hipotética 'id_v3' (tomando únicamente el último número de 'id') del DataFrame 'sucursal':
1162

• Totales de valores únicos en la columna hipotética 'id_v1_v3' (tomando la combinación del primer y el último número de 'id') del DataFrame 'sucursal':
2323

• Totales de valores únicos en la columna hipotética 'id_v2_v3' (tomando la combinación del segundo y el último número de 'id') del DataFrame 'sucursal':
1620


Ninguna de las subcombinaciones de los números en el 'id' de 'sucursal' nos identifica los registros de manera única así que optaremos por conservar el 'id' como viene presentado.

Ahora procedemos a revisar si todos los valores de la columna 'sucursal_id2' de las tablas 'precio' corresponden a un valor en la columna 'id' de 'sucursal'.

In [49]:
# Revisamos si los valores de la columna 'sucursal_id2' diferentes de '0' hacen referencia a un valor en de 'id' en el Dataframe 'sucursal'

# Revisamos sus posibles longitudes
suc_id2_lengths = set()

suc_id2_1car = []

for x in ps_2020:
    for y in ps_2020[x].sucursal_id2.unique():
        suc_id2_lengths.add(len(y))
        if len(y) == 1:
            suc_id2_1car.append(y)

print(f"En la columna 'sucursal_id2' de las tablas 'precio' encontramos valores de las siguientes longitudes: {suc_id2_lengths}")
print(f'Los valores de un sólo caracter son: {suc_id2_1car}')
print(f"\nEn la colimna 'id' de la tabla 'sucursal' encontramos valores de las siguientes longitudes: {sid_lengths}")

En la columna 'sucursal_id2' de las tablas 'precio' encontramos valores de las siguientes longitudes: {1, 5, 6, 7, 8, 9, 10}
Los valores de un sólo caracter son: ['0', '0']

En la colimna 'id' de la tabla 'sucursal' encontramos valores de las siguientes longitudes: {5, 6, 7, 8, 9, 10}


Dado que los únicos valores de un solo caracter son los '0', las longitudes se corresponden. Pasamos a revisar si los valores de cada columna 'sucursal_id2' se encuentran en la columna 'id' del DataFrame 'sucursal'.

In [50]:
# Creamos una lista (sid) con los valores de la columna 'id' de la tabla sucursal
sid = []
for x in dims['sucursal'].id.unique():
    sid.append(x)
print(len(sid))

2333


In [51]:
# El siguiente código revisa, para todas los DataFrames de precios, si los valores de 'sucursal_id2' se encuentran en la lista 'sid' recién creada.

sid_checklist = {}
sid_problems_found = {}
for x in ps_2020:
    len_x = ps_2020[x].shape[0]
    sid_checklist[x] = False
    sid_problems_found[x] = []
    for y in ps_2020[x].sucursal_id2.unique():
        if (y != '0'):
            if y not in sid:
                sid_checklist[x] = True
                sid_problems_found[x].append(y)
                #print(f'Problem found: (df:{x}) (pos:{count}) (value:{y}) (dtype:{type(y)})')

for x in sid_checklist:
    print(f"• DataFrame {x}:\n\tFound problems: {sid_checklist[x]} ({len(sid_problems_found[x])})")

• DataFrame 2020-04-26:
	Found problems: False (0)
• DataFrame 2020-04-19:
	Found problems: True (39)
• DataFrame 2020-04-13:
	Found problems: True (6)
• DataFrame 2020-05-03:
	Found problems: True (7)
• DataFrame 2020-05-18:
	Found problems: True (4)


In [52]:
print("\nValores problemáticos de 'sucursal_id' por DataFrame:")
for x in sid_problems_found:
    print(f'\n• DataFrame {x}:\n{sid_problems_found[x]}')


Valores problemáticos de 'sucursal_id' por DataFrame:

• DataFrame 2020-04-26:
[]

• DataFrame 2020-04-19:
['10-1-2029', '9-2-1939', '25-1-2001', '13-1-1939', '13-1-1962', '6-1-2009', '6-2-2021', '10-1-2006', '10-1-2018', '18-1-2005', '5-1-2003', '7-1-1937', '7-1-1948', '10-1-1946', '10-1-2026', '10-1-1933', '10-1-1948', '10-1-1953', '10-1-1954', '10-1-1944', '13-1-1952', '14-1-2009', '6-1-2004', '6-2-2002', '10-1-1955', '9-2-1950', '6-1-2026', '20-1-2001', '17-1-263', '29-1-2007', '17-1-285', '22-1-2017', '3-1-1962', '7-1-1935', '12-1-1999', '29-1-2005', '17-1-101', '12-1-1940', '65-1-315']

• DataFrame 2020-04-13:
['20-1-4', '17-1-254', '17-1-46', '17-1-252', '17-1-198', '17-1-178']

• DataFrame 2020-05-03:
['17-1-7', '22-1-11', '65-1-317', '19-1-01201', '19-1-02903', '19-1-03235', '19-1-30977']

• DataFrame 2020-05-18:
['1-1-12', '17-1-165', '17-1-122', '22-1-23']


In [53]:
# Revisamos cuántas apariciones tienen los valores problemáticos las columnas 'sucursal_id' en los DataFrames de 'precios'
sid_a_remplazar = []
for x in sid_problems_found:
    for y in sid_problems_found[x]:
        sid_a_remplazar.append(y)
sid_ar_count = 0
for x in ps_2020:
    for y in ps_2020[x].sucursal_id2:
        if y in sid_a_remplazar:
            sid_ar_count += 1
print(f'El total de productos a remplazar en los {len(ps_2020.keys())} DataFrames es de {sid_ar_count}, de un total de {tot_reg} ({round(100*(sid_ar_count/tot_reg),2)}%).')


El total de productos a remplazar en los 5 DataFrames es de 124709, de un total de 2218207 (5.62%).


Dado que tenemos una cantidad importante de registros problemáticos buscaremos si algunos de los códigos tienen un problema de formato, en específico, si al último de los 3 números separados por guiones le hace falta uno o dos '0' al comienzo, como anteriormente habíamos visto que estaban algunos de los códigos.

In [54]:
# Recordemos los códigos de sucursal_id problemáticos junto los ejemplos de valores de diferentes longitudes en la columna 'id' de sucursal
sidar_lengths = set()
for x in sid_a_remplazar:
    sidar_lengths.add(len(x))

sidar_lengths_examples = {}
for x in sidar_lengths:
    sidar_lengths_examples[str(x)] = []
for x in sid_a_remplazar:
    len_x = str(len(x))
    sidar_lengths_examples[len_x].append(x)

print("\nAlgunos ejemplos de los valores en la columna 'id' de la tabla 'sucursal' con diferentes longitudes  (longitud: ejemplos):\n")
for x in sid_lengths_examples:
    print(f'{x}: {sample(sid_lengths_examples[x],5)}')

print("\nClasificación de los valores de 'sucursal_id' a remplazar (sidar) (longitud: valores):\n")
for x in sidar_lengths_examples:
    print(f'{x}: {sidar_lengths_examples[x]}')


Algunos ejemplos de los valores en la columna 'id' de la tabla 'sucursal' con diferentes longitudes  (longitud: ejemplos):

5: ['9-2-1', '3-1-3', '8-1-7', '6-1-7', '6-1-9']
6: ['44-1-1', '13-1-2', '13-1-5', '7-1-14', '6-1-12']
7: ['2-1-266', '2-2-235', '13-1-97', '15-1-92', '13-1-58']
8: ['10-3-419', '10-3-616', '15-1-511', '10-3-481', '10-3-307']
9: ['11-2-1039', '11-5-1005', '23-1-6278', '15-1-1525', '15-1-1506']
10: ['19-1-03228', '19-1-01317', '19-1-01641', '19-1-02626', '19-1-02697']

Clasificación de los valores de 'sucursal_id' a remplazar (sidar) (longitud: valores):

6: ['20-1-4', '17-1-7', '1-1-12']
7: ['17-1-46', '22-1-11', '22-1-23']
8: ['9-2-1939', '6-1-2009', '6-2-2021', '5-1-2003', '7-1-1937', '7-1-1948', '6-1-2004', '6-2-2002', '9-2-1950', '6-1-2026', '17-1-263', '17-1-285', '3-1-1962', '7-1-1935', '17-1-101', '65-1-315', '17-1-254', '17-1-252', '17-1-198', '17-1-178', '65-1-317', '17-1-165', '17-1-122']
9: ['10-1-2029', '25-1-2001', '13-1-1939', '13-1-1962', '10-1-200

Agregaremos a los valores de 'sid_a_remplazar' que tengan 9 dígitos un 0 a la izquierda del tercer número para ver si de esta manera aparecen en la columna 'id' de la tabla 'sucursal'.

In [55]:
# Creamos la nueva lista de códigos
sidar_new = []
for x in sid_a_remplazar:
    if len(x) == 9:
        split_sidar = x.split('-')
        new_sid = split_sidar[0]+'-'+split_sidar[1]+'-0'+split_sidar[2]
        sidar_new.append(new_sid)
    else:
        sidar_new.append(x)
print(sidar_new)
print(len(sidar_new))

['10-1-02029', '9-2-1939', '25-1-02001', '13-1-01939', '13-1-01962', '6-1-2009', '6-2-2021', '10-1-02006', '10-1-02018', '18-1-02005', '5-1-2003', '7-1-1937', '7-1-1948', '10-1-01946', '10-1-02026', '10-1-01933', '10-1-01948', '10-1-01953', '10-1-01954', '10-1-01944', '13-1-01952', '14-1-02009', '6-1-2004', '6-2-2002', '10-1-01955', '9-2-1950', '6-1-2026', '20-1-02001', '17-1-263', '29-1-02007', '17-1-285', '22-1-02017', '3-1-1962', '7-1-1935', '12-1-01999', '29-1-02005', '17-1-101', '12-1-01940', '65-1-315', '20-1-4', '17-1-254', '17-1-46', '17-1-252', '17-1-198', '17-1-178', '17-1-7', '22-1-11', '65-1-317', '19-1-01201', '19-1-02903', '19-1-03235', '19-1-30977', '1-1-12', '17-1-165', '17-1-122', '22-1-23']
56


In [56]:
# Corroboramos las longitudes de los nuevos códigos (ya no debe haber códigos de 9 caracteres)
sidar_new_lengths = set()
for x in sidar_new:
    sidar_new_lengths.add(len(x))
print(sidar_new_lengths)

{8, 10, 6, 7}


In [57]:
# Revisamos cuántos de los nuevos códigos se encuentran en la columna 'id' de la tabla 'sucursal'
good_new_sidar = 0
for x in sidar_new:
    if x in dims['sucursal'].id.unique():
        good_new_sidar += 1
print(f"Un total de {good_new_sidar} de los nuevos códigos nos sirven para hacer referencia a algún valor de la columna 'id' de la tabla 'sucursal'")

Un total de 0 de los nuevos códigos nos sirven para hacer referencia a algún valor de la columna 'id' de la tabla 'sucursal'


La transformación de los códigos no nos sirvió para hacer las referencias a la tabla 'sucursal'.

Optaremos por notificar de esta situación al cliente y transformar los valores problemáticos de las columnas 'sucursal_id2' a '0'.

In [58]:
# Definimos la función para llevar a cabo la transformación
def suc_id_problem_2_zero(registro):
    if registro in sid_a_remplazar:
        return '0'
    else:
        return registro

In [59]:
for x in ps_2020:
    ps_2020[x]['sucursal_id_ok'] = ps_2020[x]['sucursal_id2'].apply(suc_id_problem_2_zero)

### Ahora limpiaremos un poco las tablas *dimensión*

In [84]:
dims['producto'].tail(5)

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
72023,9006234000107,SPEED,Bebida Energizante Speed Unlimited 250 Ml,250.0 ml,None,None,None
72024,9006234000114,SPEED,Bebida Energizante Speed Unlimited Pack 4 Un 1 Lt,1.0 lt,None,None,None
72025,9012200001245,MILKA,Chocolate Milka Noisette 300 Gr,300.0 gr,None,None,None
72026,9012200872739,MILKA,Chocolate Milka Choco swing 300 Gr,300.0 gr,None,None,None
72027,9044400841000,PEZ,Pastillero Lheritier Pez en Blister 4 Un,4.0 un,None,None,None
72028,9044400841017,SIN MARCA,Pastillas Nenas Pez 25.5 Gr,25.5 gr,None,None,None
72029,9044400841024,PEZ,Pastillero y Recarga Pez Cars 1 Un,1.0 un,None,None,None
72030,9312631127591,DILMAH,Te en Saquitos Variety Pack Dilmah 25 Un,25.0 un,None,None,None
72031,9312631143560,DILMAH,Te en Saquitos Variety Frutal Dilmah 20 Un,20.0 un,None,None,None
72032,9312631144451,DILMAH,Te Green en Saquitos Variety Pack Dilmah 20 Un,20.0 un,None,None,None


Borraremos las columnas de categoría de la tabla 'producto' pues no tenemos información para llenarlas. 
Se trato de buscar si había sido un problema al descomprimir el archivo .parquet pero en todos los visualizadores de archivos parquet en línea utilizados para ver las columnas, éstas seguían saliendo sin datos.

In [92]:
dims['producto_ok'] = dims['producto'].drop(['categoria1','categoria2','categoria3'], axis=1)

In [93]:
dims['producto_ok'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72038 entries, 0 to 72037
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            72038 non-null  object
 1   marca         72036 non-null  object
 2   nombre        72036 non-null  object
 3   presentacion  72036 non-null  object
dtypes: object(4)
memory usage: 2.2+ MB


Podemos ver que cada una de las columnas 'marca', 'nombre' y 'presentacion' tiene dos valores faltantes. Procedemos a llenarlos.

In [94]:
dims['producto_ok'].marca.fillna('SIN MARCA', inplace=True)
dims['producto_ok'].nombre.fillna('SIN NOMBRE', inplace=True)
dims['producto_ok'].presentacion.fillna('SIN DATO', inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72038 entries, 0 to 72037
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            72038 non-null  object
 1   marca         72038 non-null  object
 2   nombre        72038 non-null  object
 3   presentacion  72038 non-null  object
dtypes: object(4)
memory usage: 2.2+ MB


In [96]:
# Agregamos el registro que representa el id faltante en las tablas (producto_id='0')
pid_err_val0 = ['0000000000000', 'SIN MARCA', 'SIN NOMBRE', 'SIN DATO']
dims['producto_ok'].loc[len(dims['producto_ok'])] = pid_err_val0
dims['producto_ok'].iloc[-1]

id              0000000000000
marca               SIN MARCA
nombre             SIN NOMBRE
presentacion         SIN DATO
Name: 72038, dtype: object

In [97]:
dims['producto_ok'].tail(5)

,id,marca,nombre,presentacion
72034,9795403001143,MAYO,Mini Pizzetas Mayo 12 Un,12.0 un
72035,9990385651922,DANA,Te Negro en Hebras Lata Dana 50 Gr,50.0 gr
72036,9990385651939,DANA,Te Verde en Hebras Lata Dana 50 Gr,50.0 gr
72037,9990385651946,DANA,Yerba Mate Aromatizada Lata Dana 150 Gr,150.0 gr
72038,0000000000000,SIN MARCA,SIN NOMBRE,SIN DATO


In [99]:
dims['producto_ok'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72039 entries, 0 to 72038
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            72039 non-null  object
 1   marca         72039 non-null  object
 2   nombre        72039 non-null  object
 3   presentacion  72039 non-null  object
dtypes: object(4)
memory usage: 2.7+ MB


In [100]:
dims['producto_ok'].duplicated().value_counts()

False    72039
dtype: int64

Ya no tenemos datos faltantes en la tabla 'producto_ok' y registros duplicados.

In [65]:
dims['sucursal'].head(15)

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado
5,10-1-12,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Justo,Av. Don Bosco 2680,-34.664628,-58.597356,San Justo,Hipermercado
6,10-1-123,10,1,Hipermercado Carrefour,INC S.A.,AR-J,San Juan,Gral. Acha 32,-31.534016,-68.524744,San Juan,Supermercado
7,10-1-128,10,1,Hipermercado Carrefour,INC S.A.,AR-U,Comodoro Rivadavia,Pellegrini 851,-45.861562,-67.479968,Comodoro Rivadavia,Hipermercado
8,10-1-136,10,1,Hipermercado Carrefour,INC S.A.,AR-R,General Roca,25 De Mayo 622,-39.030326,-67.573775,General Roca,Supermercado
9,10-1-139,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Olavarría,Rivadavia 2846,-36.893694,-60.321650,Olavarría,Hipermercado


In [101]:
# Agregamos los registros para que los valores faltantes y errados de la coluna 'sucursal_id' tengan una referencia
sid_err_val0 = ['0',0,0,'SIN DATO','SIN DATO','SIN DATO','SIN DATO','SIN DATO',0,0,'SIN NOMBRE','SIN DATO']
sid_err_val1 = ['1',0,0,'SIN DATO','SIN DATO','SIN DATO','SIN DATO','SIN DATO',0,0,'SIN NOMBRE','SIN DATO']

dims['sucursal'].loc[len(dims['sucursal'])] = sid_err_val0
dims['sucursal'].loc[len(dims['sucursal'])] = sid_err_val1

dims['sucursal'].tail(5)

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
2330,9-3-5961,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-C,CIUDAD AUTONOMA BUENOS AIRES,Avenida Santa Fe 4950,-34.5772,-58.4300,Jumbo Av. Santa Fé,Supermercado
2331,9-3-628,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-B,SAN FERNANDO,Avenida Del Libertador Gral San Martin 2271,-34.4469,-58.5457,Jumbo San Fernando,Supermercado
2332,9-3-662,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-B,DEL VISO,Acceso Pilar Norte 0,-34.4360,-58.8080,Jumbo Paseo del Pilar,Supermercado
2333,0,0,0,SIN DATO,SIN DATO,SIN DATO,SIN DATO,SIN DATO,0.0000,0.0000,SIN NOMBRE,SIN DATO
2334,1,0,0,SIN DATO,SIN DATO,SIN DATO,SIN DATO,SIN DATO,0.0000,0.0000,SIN NOMBRE,SIN DATO


In [102]:
dims['sucursal'].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2335 entries, 0 to 2334
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   2335 non-null   object 
 1   comercioId           2335 non-null   int64  
 2   banderaId            2335 non-null   int64  
 3   banderaDescripcion   2335 non-null   object 
 4   comercioRazonSocial  2335 non-null   object 
 5   provincia            2335 non-null   object 
 6   localidad            2335 non-null   object 
 7   direccion            2335 non-null   object 
 8   lat                  2335 non-null   float64
 9   lng                  2335 non-null   float64
 10  sucursalNombre       2335 non-null   object 
 11  sucursalTipo         2335 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 237.1+ KB


In [103]:
dims['sucursal'].duplicated().value_counts()

False    2335
dtype: int64

Podemos ver que la tabla 'sucursal' tampoco tiene valores faltantes ni registros duplicados así que procedemos a alistar las tablas para subirlas a la base de datos.

### Preparando los CSV

Habiendo llevado a cabo esta última transformación podemos seguir a concatenar las tablas de 'precios', crando primero una columna con la fecha de los registros para no perder la separación que tenían las tablas.

In [112]:
# Concatenamos las tablas de precios en un solo DataFrame (ps_2020_ok)
dfs_to_concat = []
for x in ps_2020:
    print(f'{x}:\n{ps_2020[x].duplicated().value_counts()}')
    dfs_to_concat.append(ps_2020[x])
#print(len(dfs_to_concat))
ps_2020_ok = pd.concat(dfs_to_concat, axis=0)

print(ps_2020_ok.shape, ps_2020_ok.columns)
print(f'ps_2020_ok:\n{ps_2020_ok.duplicated().value_counts()}')

2020-04-26:
False    474692
dtype: int64
2020-04-19:
False    458543
dtype: int64
2020-04-13:
False    472134
dtype: int64
2020-05-03:
False    397734
dtype: int64
2020-05-18:
False    415104
dtype: int64
(2218207, 11) Index(['precio', 'producto_id', 'sucursal_id', 'fecha', 'precio_ok',
       'producto_id2', 'producto_id3', 'producto_id4', 'producto_id_ok',
       'sucursal_id2', 'sucursal_id_ok'],
      dtype='object')
ps_2020_ok:
False    2218207
dtype: int64


In [113]:
# Eliminamos las columnas que no necesitamos del DataFrame final
ps_2020_ok.drop(['producto_id','producto_id2','producto_id3','producto_id4','sucursal_id','sucursal_id2','precio',], axis=1, inplace=True)
print(ps_2020_ok.shape, ps_2020_ok.columns)
print(f'Valores duplicados en ps_2020_ok:\n{ps_2020_ok.duplicated().value_counts()}')

(2218207, 4) Index(['fecha', 'precio_ok', 'producto_id_ok', 'sucursal_id_ok'], dtype='object')
ps_2020_ok:
False    2208830
True        9377
dtype: int64


No está muy claro por qué quedan valores repetidos tras eliminar las columnas, pero al ser tan pocos con respecto al total, los eliminaremos.

In [114]:
ps_2020_ok.drop_duplicates(inplace=True)
print(f'Valores duplicados en ps_2020_ok:\n{ps_2020_ok.duplicated().value_counts()}')

Valores duplicados en ps_2020_ok:
False    2208830
dtype: int64


In [70]:
# Cambiamos el orden de las columnas
cols = ['precio_ok', 'producto_id_ok', 'sucursal_id_ok', 'fecha']
ps_2020_ok = ps_2020_ok[cols]
print(ps_2020_ok.columns)

Index(['precio_ok', 'producto_id_ok', 'sucursal_id_ok', 'fecha'], dtype='object')


In [71]:
# Renombramos las columnas
ps_2020_ok.rename(columns = {'precio_ok':'precio','producto_id_ok':'producto_id','sucursal_id_ok':'sucursal_id'}, inplace = True)
print(ps_2020_ok.columns)

Index(['precio', 'producto_id', 'sucursal_id', 'fecha'], dtype='object')


In [119]:
# Guardamos los DataFrames a exportar en un diccionario
df_2_export = {'precio_semanal_2020':ps_2020_ok, 'producto': dims['producto_ok'], 'sucursal':dims['sucursal']}

In [120]:
# Revisamos la información de los DataFrames finales
for x in df_2_export:
    print(f'-----{x}-----')
    print(df_2_export[x].info())

-----precio_semanal_2020-----
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2208830 entries, 0 to 415292
Data columns (total 4 columns):
 #   Column          Dtype         
---  ------          -----         
 0   fecha           datetime64[ns]
 1   precio_ok       float64       
 2   producto_id_ok  object        
 3   sucursal_id_ok  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 84.3+ MB
None
-----producto-----
<class 'pandas.core.frame.DataFrame'>
Int64Index: 72039 entries, 0 to 72038
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            72039 non-null  object
 1   marca         72039 non-null  object
 2   nombre        72039 non-null  object
 3   presentacion  72039 non-null  object
dtypes: object(4)
memory usage: 2.7+ MB
None
-----sucursal-----
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2335 entries, 0 to 2334
Data columns (total 12 columns):
 #   Column     

In [121]:
# Obtenemos la información de los DataFrames finales
for x in df_2_export:
    print(f'\n-----DataFrame: {x}-----\nShape: {df_2_export[x].shape}\n\n• Valores nulos:\n{df_2_export[x].isnull().sum()}\n\n• Registros duplicados:\n{df_2_export[x].duplicated().value_counts()}\n')


-----DataFrame: precio_semanal_2020-----
Shape: (2208830, 4)

• Valores nulos:
fecha             0
precio_ok         0
producto_id_ok    0
sucursal_id_ok    0
dtype: int64

• Registros duplicados:
False    2208830
dtype: int64


-----DataFrame: producto-----
Shape: (72039, 4)

• Valores nulos:
id              0
marca           0
nombre          0
presentacion    0
dtype: int64

• Registros duplicados:
False    72039
dtype: int64


-----DataFrame: sucursal-----
Shape: (2335, 12)

• Valores nulos:
id                     0
comercioId             0
banderaId              0
banderaDescripcion     0
comercioRazonSocial    0
provincia              0
localidad              0
direccion              0
lat                    0
lng                    0
sucursalNombre         0
sucursalTipo           0
dtype: int64

• Registros duplicados:
False    2335
dtype: int64



In [76]:
#De mysql a data frame
try:
    mydb = connection.connect(host="localhost", database = '',user="root", passwd="root",use_pure=True)
    query = "Select * from studentdetails;"
    result_dataFrame = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)